## Beam conversion from Bigquery to TF Records

In this notebook we use Apache Beam to convert to tfrecords
The applications can be found in `beam_candidates` and `beam_training` for candidate generation and training

In [1]:
# !pip install --upgrade 'apache-beam[gcp]' --user

In [2]:
import os 

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf

### Set variables

In [3]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
# BUCKET_NAME = f"{PROJECT_ID}-tfrs-retrieval"
# config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
# print(config.n)
# exec(config.n)

In [4]:
import time
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)

# TODO - parameterize
REGION = 'us-central1'
NETWORK = 'ucaip-haystack-vpc-network'
BUCKET_NAME = 'spotify-data-regimes'

BQ_DATASET = 'a_spotify_hack_v2'
BQ_TABLE_TRAIN = 'train_flat_last_15_v6' # TODO
BQ_TABLE_VALID = 'train_flat_valid_last_15_v6'
BQ_TABLE_CANDIDATES = 'candidates_v9'

MAX_PLAYLIST_LENGTH = 15

VERSION= "jtv15-8m" # version tag for dataflow pipeline

# gcs subfolders - destinations for processed data: f'gs://{BUCKET_NAME}/{VERSION}/{___PREFIX}'
CANDIDATE_PREFIX = 'candidates'
TRAIN_DIR_PREFIX = 'train'
VALID_DIR_PREFIX = 'valid'

In [5]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://spotify-data-regimes/jtv1-candidates/
                                 gs://spotify-data-regimes/jtv1/
                                 gs://spotify-data-regimes/jtv10/
                                 gs://spotify-data-regimes/jtv11/
                                 gs://spotify-data-regimes/jtv12/
                                 gs://spotify-data-regimes/jtv13-lite/
                                 gs://spotify-data-regimes/jtv14-8m/
                                 gs://spotify-data-regimes/jtv15-8m/
                                 gs://spotify-data-regimes/jtv2/
                                 gs://spotify-data-regimes/jtv4/
                                 gs://spotify-data-regimes/jtv5/
                                 gs://spotify-data-regimes/jtv8/
                                 gs://spotify-data-regimes/jtv9demo/
                                 gs://spotify-data-regimes/test/
                                 gs://spotify-data-regimes

# Run Dataflow to convert BQ to TFrecords

Candidate generation can be found in `beam_candidates`
Training and Validation generation can be found in `beam_training`

Usage:
* Candidate generation

> `beam_candidates\python3 main.py`
   
* Training generation
  
> `beam_training\python3 main-train.py <BQ_table> <gcs data subfolder> <desired partition size MB> <BQ dataset size MB> <version tag>`

In [4]:
!tree beam_training

beam_training
├── README.MD
├── __init__.py
├── main-train.py
├── setup.py
└── train_pipeline
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-37.pyc
    │   └── train_pipe.cpython-37.pyc
    └── train_pipe_shape.py

2 directories, 8 files


In [5]:
# os.chdir('/home/jupyter/jw-repo/spotify_mpd_two_tower')
os.getcwd()

'/home/jupyter/jw-repo/spotify_mpd_two_tower'

In [6]:
%cd beam_training

/home/jupyter/jw-repo/spotify_mpd_two_tower/beam_training


### Validation set

In [7]:
TARGET_SHARD_SIZE_MB_VALID = 250
TOTAL_MB_VALID = 500
NUM_TF_RECORDS = int(TOTAL_MB_VALID) // int(TARGET_SHARD_SIZE_MB_VALID)
NUM_TF_RECORDS

2

In [8]:
start_time = time.time()

# ! python3 main-train.py $BQ_TABLE_VALID $VALID_DIR_PREFIX $TARGET_SHARD_SIZE_MB_VALID $TOTAL_MB_VALID $VERSION $BUCKET_NAME $REGION $PROJECT_ID $NETWORK $BQ_DATASET

! python3 main-train.py $PROJECT_ID $NETWORK $REGION $VERSION $BUCKET_NAME $VALID_DIR_PREFIX $TOTAL_MB_VALID $TARGET_SHARD_SIZE_MB_VALID $BQ_DATASET $BQ_TABLE_VALID

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 2
2023-01-27 16:12:37.003866: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-jtv15-8m-230127-161240, labels=None, no_auth=False, project=hybrid-vertex, re

### Tain set

In [10]:
TARGET_SHARD_SIZE_MB_TRAIN = 2000
TOTAL_MB_TRAIN = 44_000
NUM_TF_RECORDS = int(TOTAL_MB_TRAIN) // int(TARGET_SHARD_SIZE_MB_TRAIN)
NUM_TF_RECORDS

22

In [11]:
start_time = time.time()

! python3 main-train.py $PROJECT_ID $NETWORK $REGION $VERSION $BUCKET_NAME $TRAIN_DIR_PREFIX $TOTAL_MB_TRAIN $TARGET_SHARD_SIZE_MB_TRAIN $BQ_DATASET $BQ_TABLE_TRAIN

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 22
2023-01-27 16:33:09.276993: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-jtv15-8m-230127-163312, labels=None, no_auth=False, project=hybrid-vertex, r

# Now export the candidates

**TODO:** clean-up section

In [13]:
%cd ../beam_candidates

/home/jupyter/jw-repo/spotify_mpd_two_tower/beam_candidates


In [15]:
! python3 main.py $PROJECT_ID $NETWORK $REGION $VERSION $BUCKET_NAME $CANDIDATE_PREFIX $BQ_DATASET $BQ_TABLE_CANDIDATES

2023-01-27 17:05:42.868226: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-jtv15-8m-230127-170542, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_accoun

# Test output

## Candidates

### Candidate tower features

In [16]:
candidate_features = {
    "track_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),            
    "track_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "album_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),           
    "album_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
    "duration_ms_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
    "track_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
    "artist_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "artist_genres_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_followers_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # new
    # "track_pl_titles_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_danceability_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_energy_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_key_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_loudness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_mode_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_speechiness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_acousticness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_instrumentalness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_liveness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_valence_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_tempo_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "time_signature_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
}

### Candidate files

In [19]:
# candidate_files = []
# for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{CANDIDATE_PREFIX}', delimiter="/"):
#     candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

# candidate_files = f'gs://{BUCKET_NAME}/{VERSION}/{CANDIDATE_PREFIX}/candidates-00000-of-00001.tfrecords'
candidate_files = f'gs://{BUCKET_NAME}/{VERSION}/candidates-00000-of-00001.tfrecords'
    
candidate_dataset = tf.data.TFRecordDataset(candidate_files)

def parse_candidate_tfrecord_fn(example):
    example = tf.io.parse_single_example(
        example, 
        features=candidate_features
    )
    return example

parsed_candidate_dataset = candidate_dataset.map(parse_candidate_tfrecord_fn)

In [20]:
from pprint import pprint

for x in parsed_candidate_dataset.batch(1).take(1):
    pprint(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"Nim's Island"], dtype=object)>,
 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:3s9jAEzWINEIDxqbtqkli3'], dtype=object)>,
 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([72135.], dtype=float32)>,
 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"'british soundtrack', 'hollywood', 'orchestral soundtrack', 'scorecore', 'soundtrack'"],
      dtype=object)>,
 'artist_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Patrick Doyle'], dtype=object)>,
 'artist_pop_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([61.], dtype=float32)>,
 'artist_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:artist:1W42coQfIlt6btgqpfJWYQ'], dtype=object)>,
 'duration_ms_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([242320.], dtype=float32)>,
 'time_signature_can': <tf.Tensor: shape=(1,), dtyp

## Train & Valid datasets

In [6]:
MAX_PLAYLIST_LENGTH = 15

### train & valid features

In [11]:
feats = {
    # ===================================================
    # candidate track features
    # ===================================================
    "track_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),            
    "track_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "album_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),           
    "album_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
    "duration_ms_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
    "track_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
    "artist_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "artist_genres_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "artist_followers_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # "track_pl_titles_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_danceability_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_energy_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_key_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_loudness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_mode_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    "track_speechiness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_acousticness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_instrumentalness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_liveness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_valence_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "track_tempo_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    "time_signature_can": tf.io.FixedLenFeature(dtype=tf.string, shape=()),
    
    # ===================================================
    # summary playlist features
    # ===================================================
    "pl_name_src" : tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
    'pl_collaborative_src' : tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
    # 'num_pl_followers_src' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
    'pl_duration_ms_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
    'num_pl_songs_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()),  # num_pl_songs_new | n_songs_pl_new
    'num_pl_artists_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
    'num_pl_albums_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
    # 'avg_track_pop_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
    # 'avg_artist_pop_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    # 'avg_art_followers_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
    
    # ===================================================
    # ragged playlist features
    # ===================================================
    # bytes / string
    "track_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "track_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "artist_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "artist_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)),
    "album_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "album_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "artist_genres_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    # "tracks_playlist_titles_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)),
    
    # Float List
    "duration_ms_songs_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)), 
    "track_pop_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)), 
    "artist_pop_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)), 
    "artists_followers_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_danceability_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_energy_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_key_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
    "track_loudness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_mode_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_speechiness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_acousticness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_instrumentalness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_liveness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)), 
    "track_valence_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)),
    "track_tempo_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(MAX_PLAYLIST_LENGTH,)), 
    "time_signature_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)), 
}


### Valid files

In [38]:
valid_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{VERSION}/{VALID_DIR_PREFIX}/', delimiter="/"):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
valid = tf.data.TFRecordDataset(valid_files)

def parse_tfrecord(example):
    example = tf.io.parse_single_example(
        example, 
        features=feats
    )
    return example

valid_parsed = valid.map(parse_tfrecord)

In [42]:
# valid_parsed

In [41]:
for x in valid_parsed.batch(1).take(1):
    print(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Lolly'], dtype=object)>, 'album_name_pl': <tf.Tensor: shape=(1, 15), dtype=string, numpy=
array([[b'9', b'In My Mind', b'Nervous System',
        b'Nervous (The Ooh Song)', b'Fast Car', b'Wedding Bells EP',
        b'DARKNESS AND LIGHT', b'Night & Day',
        b'A Head Full Of Dreams Tour Edition', b'FUTURE', b'Oh My My',
        b'GALLERY', b'Once In a While', b'By Your Side',
        b'Double Vision']], dtype=object)>, 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:0Jy2FoyFXxrHi6XIa8cHzU'], dtype=object)>, 'album_uri_pl': <tf.Tensor: shape=(1, 15), dtype=string, numpy=
array([[b'spotify:album:0JjoGwsBjce2mgHRliH0VN',
        b'spotify:album:0YOVfyE9xg8noB5rDNLtm4',
        b'spotify:album:22aqlgX39cWQMWazg9qwuv',
        b'spotify:album:6jfuBG9u0Au2nWXGXTXmux',
        b'spotify:album:5jfuIbTeaLhBZxsCVv3QyM',
        b'spotify:album:4IzStrE9iX01yM5gUUAe74',
        b'spotify:

### Train files

In [8]:
train_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{VERSION}/{TRAIN_DIR_PREFIX}/', delimiter="/"):
    if '.tfrecords' in blob.name:
        train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
train = tf.data.TFRecordDataset(train_files)

def parse_tfrecord(example):
    example = tf.io.parse_single_example(
        example, 
        features=feats
    )
    return example

train_parsed = train.map(parse_tfrecord)

KeyboardInterrupt: 

In [46]:
train_parsed

In [45]:
for x in train_parsed.batch(1).take(1):
    print(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Melody Hits Vol. 7'], dtype=object)>, 'album_name_pl': <tf.Tensor: shape=(1, 15), dtype=string, numpy=
array([[b'Ahla W Ahla (Summer Edition)', b'Ayech Bi Oyouni',
        b'Sweetness of Nancy', b'Enti Mshiti',
        b'Ahla W Ahla (Summer Edition)',
        b'Top 25 DJ Mixes of Arabic Music',
        b'Ahla W Ahla (Summer Edition)', b'Best Arabic Music',
        b'Lamaallem', b'Top 25 DJ Mixes of Arabic Music',
        b'Melody Hits, Vol. 6', b'Top 25 DJ Mixes of Arabic Music',
        b'Ghaltana', b'Malyoun Bhebbik Malyoun', b'Ghammadt Einy']],
      dtype=object)>, 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:4GHKFNuSbVVcdwGtZ6AlCt'], dtype=object)>, 'album_uri_pl': <tf.Tensor: shape=(1, 15), dtype=string, numpy=
array([[b'spotify:album:5A4q1RPTVObTLih6EhG67w',
        b'spotify:album:5pfkDggECTlWNLL9Jf4orK',
        b'spotify:album:4ZC7cD2S99dPXUMKkQ6Sic',
        b'spoti